In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Layer,MaxPool2D,Dense,Input,Flatten,Conv2D,Dropout
from tensorflow.keras.utils import to_categorical
import cv2

In [4]:
data=pd.read_csv("../../Downloads/icml_face_data.csv")

In [5]:
data=pd.DataFrame(data)

In [6]:
data.iloc[:,1].unique()

array(['Training', 'PublicTest', 'PrivateTest'], dtype=object)

In [7]:
data.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [8]:
data.rename(columns={' Usage':'Usage',' emotion':'emotion',' pixels':'pixels'},inplace=True)

In [7]:
#final_D_test=data.loc[data["Usage"]=='PublicTest']
#final_D_train=data.loc[data['Usage']=='Training']

In [8]:
#final_D_train['Usage'].unique()

In [9]:
#final_D_test['Usage'].unique()

In [10]:
x_trian,y_trian,x_test,y_test=[],[],[],[]

In [11]:
for index,row in data.iterrows():
    val=row['pixels'].split(' ')
    
    if 'Training' in row['Usage']:
        x_trian.append(np.array(val,'float32'))  ### each element dtype it is float32 
        y_trian.append(row['emotion'])
    elif 'PublicTest' in row['Usage']:
        x_test.append(np.array(val,'float32'))
        y_test.append(row['emotion'])

In [15]:
#a=data['pixels'][0]
#a=a.split(' ')
#a=np.array(a,'float32')
#a

In [16]:
#a=np.array(a)
#a

array([ 70.,  80.,  82., ..., 106., 109.,  82.], dtype=float32)

In [13]:
x_trian=np.array(x_trian,'float32')            #### whole train dtype varibale is converted to float32
y_trian=np.array(y_trian,'float32')
x_test=np.array(x_test,'float32')
y_test=np.array(y_test,'float32')

In [14]:
## normalize our data between 0 to 1

In [15]:
x_trian-=np.mean(x_trian,axis=0)
x_trian/=np.std(x_trian,axis=0)

In [16]:
#x_trian[0]

In [17]:
x_test-=np.mean(x_test,axis=0)
x_test/=np.std(x_test,axis=0)

In [18]:
num_features=64
num_labels=7
batch_size=64
width=48
height=48
epochs=30

In [19]:
x_trian=x_trian.reshape(x_trian.shape[0],width,height,1)

In [20]:
x_test=x_test.reshape(x_test.shape[0],width,height,1)

In [21]:
y_trian=to_categorical(y_trian,num_classes=num_labels)
y_test=to_categorical(y_test,num_classes=num_labels)

In [22]:
####Dropout is a technique where randomly selected neurons are ignored during training. 
###  They are “dropped-out” randomly

In [23]:
in_layer=Input(shape=(x_trian.shape[1:]))

## 1 layer cnn layer

convo1=Conv2D(num_features,kernel_size=(3,3),activation='relu')(in_layer)
convo2=Conv2D(num_features,kernel_size=(3,3),activation='relu')(convo1)
pool1=MaxPool2D(pool_size=(2,2),strides=(2,2))(convo2)
drop1=Dropout(0.5)(pool1)  

## 2 layer of cnn

convo3=Conv2D(num_features,(3,3),activation='relu')(drop1)
convo4=Conv2D(num_features,(3,3),activation='relu')(convo3)
pool2=MaxPool2D(pool_size=(2,2),strides=(2,2))(convo4)
drop2=Dropout(0.5)(pool2) 

## 3 layer of cnn

convo5=Conv2D(2*num_features,(3,3),activation='relu')(drop2)
convo6=Conv2D(2*num_features,(3,3),activation='relu')(convo5)
pool3=MaxPool2D(pool_size=(2,2),strides=(2,2))(convo6)
 
flat=Flatten()(pool3)

d1=Dense(2*2*2*2*num_features,activation='relu')(flat)
drop3=Dropout(0.2)(d1)
d2=Dense(2*2*2*2*num_features,activation='relu')(drop3)
drop4=Dropout(0.2)(d2)

final=Dense(num_labels,activation='softmax')(drop4)

In [24]:
model=Model(inputs=in_layer,outputs=final)

In [25]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [26]:
model.fit(x_trian,y_trian,epochs=epochs,validation_data=(x_test,y_test),shuffle=True)

Epoch 1/30
898/898 [==============================] - 364s 405ms/step - loss: 1.7268 - accuracy: 0.2934 - val_loss: 1.5640 - val_accuracy: 0.3918
Epoch 2/30
898/898 [==============================] - 343s 382ms/step - loss: 1.5210 - accuracy: 0.4021 - val_loss: 1.4049 - val_accuracy: 0.4583
Epoch 3/30
898/898 [==============================] - 343s 382ms/step - loss: 1.4262 - accuracy: 0.4464 - val_loss: 1.3417 - val_accuracy: 0.4829
Epoch 4/30
898/898 [==============================] - 343s 382ms/step - loss: 1.3737 - accuracy: 0.4683 - val_loss: 1.3179 - val_accuracy: 0.4879
Epoch 5/30
898/898 [==============================] - 344s 384ms/step - loss: 1.3311 - accuracy: 0.4863 - val_loss: 1.2908 - val_accuracy: 0.4946
Epoch 6/30
898/898 [==============================] - 348s 387ms/step - loss: 1.3041 - accuracy: 0.4999 - val_loss: 1.2604 - val_accuracy: 0.5099
Epoch 7/30
898/898 [==============================] - 344s 383ms/step - loss: 1.2786 - accuracy: 0.5074 - val_loss: 1.2365 -

In [28]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 22, 22, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        36928 

In [30]:
fer_json=model.to_json()

In [31]:
with open("fer.json","w") as json_file:
    json_file.write(fer_json)

In [32]:
model.save_weights("fer.h5")  ## save weight in h5 format

In [27]:
#### yt model train

model=Sequential()

## 1 layer of cnn

model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu',input_shape=x_trian.shape[1:]))
model.add(Conv2D(num_features,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

## 2 layer of cnn

model.add(Conv2D(num_features,(3,3),activation='relu'))
model.add(Conv2D(num_features,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))

## 3 layer of cnn
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.5))         

model.add(Flatten())
          
model.add(Dense(2*2*2*2*num_features,activation='relu'))
model.add(Dropout(0.2))
          
model.add(Dense(2*2*2*2*num_features,activation='relu'))
model.add(Dropout(0.2))
    
model.add(Dense(7,activation='softmax'))